In [1]:
import requests
import pandas as pd
from datetime import datetime as dt
from xml.etree import ElementTree as ET
from config import userid, password
from bs4 import BeautifulSoup

In [3]:
tracking_df = pd.read_csv('TrackingList.csv')
track_list = list(tracking_df['Tracking Nos.'])
len(track_list)

121

In [5]:
batch_list = track_list[0:35]

# OK, let's build the url for one batch:
base_url = 'https://secure.shippingapis.com/ShippingAPI.dll?API=TrackV2&XML='

for number in batch_list:
    call_url = base_url + f'<TrackRequest USERID="{userid}" PASSWORD="{password}">'
    for number in batch_list:
        call_url += f'<TrackID ID="{number}"></TrackID>'
    call_url += '</TrackRequest>'

response = requests.get(call_url)
soup = BeautifulSoup(response.content, 'xml')

soup

<?xml version="1.0" encoding="utf-8"?>
<TrackResponse><TrackInfo ID="70191640000164106422"><TrackSummary>Your package is moving within the USPS network and is on track to be delivered to its final destination. It is currently in transit to the next facility.</TrackSummary><TrackDetail>In Transit to Next Facility, 04/22/2024</TrackDetail><TrackDetail>In Transit to Next Facility, 04/21/2024</TrackDetail><TrackDetail>In Transit to Next Facility, 04/20/2024</TrackDetail><TrackDetail>Arrived at USPS Regional Facility, April 18, 2024, 8:36 am, CHARLOTTE NC DISTRIBUTION CENTER</TrackDetail><TrackDetail>In Transit to Next Facility, 04/17/2024</TrackDetail><TrackDetail>Arrived at USPS Regional Facility, April 16, 2024, 12:34 am, COPPELL TX DISTRIBUTION CENTER</TrackDetail></TrackInfo><TrackInfo ID="70192280000060192481"><TrackSummary>Your package is moving within the USPS network and is on track to be delivered to its final destination. It is currently in transit to the next facility.</TrackSum

In [7]:
delivered_list = []

for number in batch_list:
    summary = soup.find('TrackInfo', ID=number).find('TrackSummary')
    
    if summary != None:
        # Three different phrases might indicate that the package has reached the recipient:
        condition1 = 'Your item was delivered' in summary.text
        condition2 = 'Your item has been delivered' in summary.text
        condition3 = 'Your item was picked up' in summary.text

        if condition1 | condition2 | condition3:
            delivered_list.append(number)

delivered_list

[]